# Sobre a modelagem

As bibliotecas *Numpy* e *MatPlotLib* devem ser importadas para que tenhamos métodos para calcular as equações e Plotar a dinâmica, respectivamente.

O artigo apresenta a modelagem de um sistema CSTR através das seguintes equações, no estado estacionário:

$$
F_o(C_{Ao} - C_A) - VkC_A = 0 
$$ 

$$ 
\rho C_pF_o(T_o-T_j) - \lambda VkC_A - UA(T - T_j) = 0
$$

$$
\rho_j C_jF_j(T_{jo}-T_j)+UA(T - T_j) = 0
$$

Ocorre que $k = \alpha e^{E/RT}$ , o que nos leva a reescrever as duas primeiras equações como:

$$
F_o(C_{Ao} - C_A) - V \alpha e^{E/RT} C_A = 0
$$ 

$$ 
\rho C_pF_o(T_o-T_j) - \lambda V \alpha e^{E/RT} C_A - UA(T - T_j) = 0
$$

Fazemos isso para obter três equações e três variáveis para as quais devemos resolver essas equações, a saber: $T$ , $C_j$ e $T_j$. Sendo assim, temos um sistema de equações não lineares. 

É preciso notar que essas equações podem ter múltiplas soluções. Sendo assim, a resolução direta dessas equações não nos fornecerá todos os estados estacionários — como o próprio artigo argumenta —, mas bastará para os objetivos desta demonstração. Além disso, os estados estacionários dependem da temperatura de entrada do fluido, que vamos tratar como uma constate $T_o = 530°C$

Logo, o que faremos é:

* Encontrar pelo menos uma solução de estado estacionário e;
* Plotar a dinâmica do sistema utilizando método de Euler.

In [2]:
from scipy.optimize import fsolve
from math import exp
import numpy as np
import matplotlib.pyplot as plt

# Estacionário

In [26]:
# VARIAVEIS DO REATOR

CAo    = 0.5    # Concentração do fluido A na entrada;
rho    = 50    # Densidade do fluido A;
U      = 150    # Energia;
A      = 250    # Área do reator;
V      = 48    # Volume interno do Reator;
Fi     = 40    # Fluxo na saída do reator;
Fo     = 40    # Fluxo na entrada do reator;
To     = 530

# VARIAVEIS DA CAMISA

Fj     = 49.9    # Fluxo da camisa;
Vj     = 3.85    # Volume da camisa;
Cj     = 1       # Concentração do fluido refrigerante
Cp     = 0.75
rhoj   = 62.3    # Densidade do fluido refrigerante;
Toj    = 530

# CONSTANTES DA REAÇÂO

lbd     = -30000     
alfa    = 78000000000
E       = 30000
R       = 1.99

# VARIAVEIS QUE PROCURAMOS

T      = 530    # Temperatura interna do reator;
Tj     = 500    # Temperatura interna do reator;
Ca     = 0.5    # Concentração do fluido A na Saída;



import numpy as np
from matplotlib import pyplot as plt
from scipy.optimize import fsolve
from math import exp

Ea = 30000     # activation energy J/gmol
R = 1.99      # gas constant J/gmol/K
k0 = 7.8e10    # Arrhenius rate constant 1/min

# Arrhenius rate expression
def k(T):
    return k0*np.exp(-Ea/R/T)


def SistemaEquacoes(vars):
    T, Tj, Ca = vars
    k = alfa*np.exp(-E/R*T)
    eq1 = Fo * (CAo - Ca) - V * k * Ca
    eq2 = rho * Cp * Fo * (To - T) - lbd * k * Ca - U * A * (T - Tj)
    eq3 = rhoj * Cj * Fj * (Toj-Tj) + U * A * (T - Tj)
    print(eq1, eq2, eq3)
    return [eq1, eq2, eq3]


fsolve(SistemaEquacoes, [T, Tj, Ca])

0.0 -1125000.0 1218263.1
0.0 -1125000.0 1218263.1
0.0 -1125000.0 1218263.1
0.0 -1125000.3080070019 1218263.3961605788
0.0 -1124999.7206032276 1218262.7974410863
-2.980232238769531e-07 -1125000.0 1218263.1
-2.3283153183228933e-10 2.4556356947869062e-06 -2.6592092035571113e-06
0.0 0.0 0.0


array([5.3e+02, 5.3e+02, 5.0e-01])

# Dinâmica

In [27]:
def SistemaEquacoesSemK(vars):
    T, Tj, Ca = vars
    eq1 = Fo * (CAo - Ca)
    eq2 = rho * Cp * Fo * (To - T) - U * A * (T - Tj)
    eq3 = rhoj * Cj * Fj * (Toj-Tj) + U * A * (T - Tj)
    print(eq1, eq2, eq3)
    return [eq1, eq2, eq3]

fsolve(SistemaEquacoes, [T, Tj, Ca])

# T = np.linspace(0, 700)
# plt.plot(T, )
# plt.xlabel('Absolute Temperature [K]')
# plt.ylabel('Rate Constant [1/min]')
# plt.title('Arrenhius Rate Constant')

0.0 -1125000.0 1218263.1
0.0 -1125000.0 1218263.1
0.0 -1125000.0 1218263.1
0.0 -1125000.3080070019 1218263.3961605788
0.0 -1124999.7206032276 1218262.7974410863
-2.980232238769531e-07 -1125000.0 1218263.1
-2.3283153183228933e-10 2.4556356947869062e-06 -2.6592092035571113e-06
0.0 0.0 0.0


array([5.3e+02, 5.3e+02, 5.0e-01])